In [1]:
import os
import re
from dotenv import load_dotenv
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Load environment variables
load_dotenv()


# Watsonx chat function
def watsonx_llm(messages):
    """
    Generate a chat completion using IBM Watsonx models.

    Parameters:
    - messages (list of dict): The messages for the chat session.
    - model_size (int, optional): Specifies the model size. Default is 8.

    Returns:
    - str: The generated response content.
    """
    model_id = f"meta-llama/llama-3-1-8b-instruct"

    # Fetch environment variables
    api_key = os.getenv("WX_KEY")
    url = os.getenv("WX_URL")
    project_id = os.getenv("WX_PID")

    if not all([api_key, project_id, model_id]):
        raise Exception(
            "Missing required environment variables: WX_KEY, WX_URL, or WX_PID."
        )

    # Adjusted parameters
    params = {
        GenParams.DECODING_METHOD: "sample",
        GenParams.MAX_NEW_TOKENS: 256,
        GenParams.TEMPERATURE: 0.0,
        GenParams.STOP_SEQUENCES: ["<|eot_id|>", "<|eom_id|>"],
    }

    # Setup credentials
    credentials = Credentials(api_key=api_key, url=url)

    # Instantiate ModelInference
    model_inference = ModelInference(
        model_id=model_id, params=params, credentials=credentials, project_id=project_id
    )

    # Call the chat method
    response = model_inference.chat(messages=messages)

    # Extract and return the generated content
    return response["choices"][0]["message"]["content"]


# Define the ReAct Agent class
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return watsonx_llm(self.messages)


# Define available actions
def calculate(what):
    try:
        return eval(what)
    except Exception as e:
        return f"Error in calculation: {e}"


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weighs 50 lbs"


def calculator(expression):
    """
    A more advanced calculator that supports complex mathematical expressions.
    """
    try:
        # Safeguard to limit the operations
        result = eval(expression, {"__builtins__": {}}, {})
        return result
    except Exception as e:
        return f"Error: {e}"


# Add tools to known actions
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
    "calculator": calculator,
}

# Initialize the prompt
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

average_dog_weight:
e.g. average_dog_weight: Collie
Returns the average weight of a dog when given the breed.

calculator:
e.g. calculator: (5 + 10) * (2 ** 3)
Runs a more advanced calculation and returns the result. It can handle complex mathematical expressions.

Example session:

Question: What is the result of (3 + 7) * 2?
Thought: I should use the calculator tool to compute the result.
Action: calculator: (3 + 7) * 2
PAUSE

You will be called again with this:

Observation: The result is 20

You then output:

Answer: The result of (3 + 7) * 2 is 20.
""".strip()

# Define the query function with ReAct loop
action_re = re.compile(r"^Action: (\w+): (.*)$")


def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return


# Example usage
question = "I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"
query(question)

/Users/charan/VSCode/GITHUB/Assets_Youtube_Videos/agentic_rag/venv/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/inference/base_model_inference.py:743: UserWarning: Parameters [decoding_method, max_new_tokens, stop_sequences] is/are not recognized and will be ignored.
  warnings.warn(


Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE

Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE

Thought: Now that I have the average weights of both breeds, I can add them together to get the combined weight.
Action: calculate: 22 + 10
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs


/Users/charan/VSCode/GITHUB/Assets_Youtube_Videos/agentic_rag/venv/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/inference/base_model_inference.py:743: UserWarning: Parameters [decoding_method, max_new_tokens, stop_sequences] is/are not recognized and will be ignored.
  warnings.warn(


Thought: Now that I know the average weight of a Border Collie, I can find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs


/Users/charan/VSCode/GITHUB/Assets_Youtube_Videos/agentic_rag/venv/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/inference/base_model_inference.py:743: UserWarning: Parameters [decoding_method, max_new_tokens, stop_sequences] is/are not recognized and will be ignored.
  warnings.warn(


Thought: Now that I have the average weights of both breeds, I can add them together to get the combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57


/Users/charan/VSCode/GITHUB/Assets_Youtube_Videos/agentic_rag/venv/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/inference/base_model_inference.py:743: UserWarning: Parameters [decoding_method, max_new_tokens, stop_sequences] is/are not recognized and will be ignored.
  warnings.warn(


Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


In [ ]:
# # Import required modules
# import os  # For interacting with operating system environment variables
# import re  # For working with regular expressions
# from dotenv import load_dotenv  # To load environment variables from a .env file
# from ibm_watsonx_ai import Credentials  # To manage API credentials for IBM Watsonx
# from ibm_watsonx_ai.foundation_models import ModelInference  # To interface with Watsonx AI models
# from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams  # Predefined parameter keys for text generation

# # Load environment variables from a .env file
# load_dotenv()

# # Define a function to interact with Watsonx LLM (Large Language Model)
# def watsonx_llm(messages):
#     """
#     Generate a chat completion using IBM Watsonx models.

#     Parameters:
#     - messages (list of dict): A list of message objects that simulate a chat session.
#       Each message contains a 'role' (e.g., user, system, assistant) and its 'content'.
    
#     Returns:
#     - str: The response content generated by the model.
#     """

#     # Specify the model ID to use, in this case, an IBM Watsonx LLaMA-based model
#     model_id = f"meta-llama/llama-3-1-8b-instruct"

#     # Fetch required environment variables for API authentication
#     api_key = os.getenv("WX_KEY")  # API key for Watsonx authentication
#     url = os.getenv("WX_URL")  # Base URL for the Watsonx service
#     project_id = os.getenv("WX_PID")  # Project ID associated with the Watsonx account

#     # Ensure all required environment variables are available; raise an error otherwise
#     if not all([api_key, project_id, model_id]):
#         raise Exception(
#             "Missing required environment variables: WX_KEY, WX_URL, or WX_PID."
#         )

#     # Define parameters for text generation
#     params = {
#         GenParams.DECODING_METHOD: "sample",  # Specify the decoding method
#         GenParams.MAX_NEW_TOKENS: 256,  # Set the maximum number of tokens to generate
#         GenParams.TEMPERATURE: 0.0,  # Temperature for deterministic responses
#         GenParams.STOP_SEQUENCES: ["<|eot_id|>", "<|eom_id|>"],  # Define stop sequences
#     }

#     # Initialize credentials object with API key and service URL
#     credentials = Credentials(api_key=api_key, url=url)

#     # Create a ModelInference instance to interact with the specified model
#     model_inference = ModelInference(
#         model_id=model_id, params=params, credentials=credentials, project_id=project_id
#     )

#     # Generate a response from the model using the provided messages
#     response = model_inference.chat(messages=messages)

#     # Return the content of the first choice in the response
#     return response["choices"][0]["message"]["content"]

# # Define a class for a ReAct agent
# class Agent:
#     def __init__(self, system=""):
#         """
#         Initialize the agent with an optional system message.
#         """
#         self.system = system  # System message for setting the agent's initial behavior
#         self.messages = []  # List to store the chat history
#         if self.system:  # If a system message is provided, add it to messages
#             self.messages.append({"role": "system", "content": system})

#     def __call__(self, message):
#         """
#         Add a user message and generate a response from the agent.
#         """
#         self.messages.append({"role": "user", "content": message})  # Add user message
#         result = self.execute()  # Generate a response
#         self.messages.append({"role": "assistant", "content": result})  # Add assistant's response
#         return result

#     def execute(self):
#         """
#         Execute the Watsonx model with the current messages.
#         """
#         return watsonx_llm(self.messages)  # Use Watsonx LLM to generate a response

# # Define actions for specific queries
# def calculate(what):
#     """
#     Perform a calculation based on the provided input string.
#     """
#     try:
#         return eval(what)  # Evaluate the input string as a Python expression
#     except Exception as e:
#         return f"Error in calculation: {e}"  # Handle errors gracefully

# def average_dog_weight(name):
#     """
#     Return the average weight of a dog based on its breed.
#     """
#     if name in "Scottish Terrier":
#         return "Scottish Terriers average 20 lbs"
#     elif name in "Border Collie":
#         return "a Border Collies average weight is 37 lbs"
#     elif name in "Toy Poodle":
#         return "a toy poodles average weight is 7 lbs"
#     else:
#         return "An average dog weighs 50 lbs"

# def calculator(expression):
#     """
#     Evaluate complex mathematical expressions securely.
#     """
#     try:
#         # Evaluate the expression with restricted built-ins for security
#         result = eval(expression, {"__builtins__": {}}, {})
#         return result
#     except Exception as e:
#         return f"Error: {e}"  # Handle errors

# # Create a dictionary of known actions for ReAct
# known_actions = {
#     "calculate": calculate,  # Simple calculator action
#     "average_dog_weight": average_dog_weight,  # Retrieve average dog weight
#     "calculator": calculator,  # Advanced calculator action
# }

# # Prompt to instruct the agent on its behavior
# prompt = """
# You run in a loop of Thought, Action, PAUSE, Observation.
# At the end of the loop you output an Answer.
# Use Thought to describe your thoughts about the question you have been asked.
# Use Action to run one of the actions available to you - then return PAUSE.
# Observation will be the result of running those actions.

# Your available actions are:

# calculate:
# e.g. calculate: 4 * 7 / 3
# Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

# average_dog_weight:
# e.g. average_dog_weight: Collie
# Returns the average weight of a dog when given the breed.

# calculator:
# e.g. calculator: (5 + 10) * (2 ** 3)
# Runs a more advanced calculation and returns the result. It can handle complex mathematical expressions.

# Example session:

# Question: What is the result of (3 + 7) * 2?
# Thought: I should use the calculator tool to compute the result.
# Action: calculator: (3 + 7) * 2
# PAUSE

# You will be called again with this:

# Observation: The result is 20

# You then output:

# Answer: The result of (3 + 7) * 2 is 20.
# """.strip()

# # Define a query function with a ReAct loop
# action_re = re.compile(r"^Action: (\w+): (.*)$")  # Regex to parse actions

# def query(question, max_turns=5):
#     """
#     Process a question in a loop using the ReAct framework.
#     """
#     i = 0  # Initialize loop counter
#     bot = Agent(prompt)  # Create an agent with the defined prompt
#     next_prompt = question  # Set the initial prompt to the question
#     while i < max_turns:  # Limit the number of turns to prevent infinite loops
#         i += 1
#         result = bot(next_prompt)  # Generate a response from the agent
#         print(result)  # Print the result
#         actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]  # Parse actions
#         if actions:  # If an action is found
#             action, action_input = actions[0].groups()  # Extract action name and input
#             if action not in known_actions:  # Check if action is recognized
#                 raise Exception(f"Unknown action: {action}: {action_input}")  # Handle unknown actions
#             print(f" -- running {action} {action_input}")  # Log the action
#             observation = known_actions[action](action_input)  # Execute the action
#             print("Observation:", observation)  # Print the observation
#             next_prompt = f"Observation: {observation}"  # Update the prompt with the observation
#         else:  # If no actions remain, terminate
#             return

# # Example usage of the query function
# question = "I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"
# query(question)
